# Security 2: HW 2 \- Exercise on RSA

In this document is the python sagemath notebook solution to the homework, this implementation would not be secure for use no proper padding PSS or OAEP, not constant time due to python optimizations, uses SHA\-1 for hashing which is weak for modern security needs, the randomness used in key generation may not be very cryptographically secure.


### As extra I made 2 other versions in C that are available here: [https://github.com/mattc\-try/rsa\-personal\-implementations](https://github.com/mattc-try/rsa-personal-implementations)

1 using standard bignum and sha from openssl, I am quite familiar with it

2 pure C implementation, with some assembly optimization if I have time to finish it by the deadline \(not tested for any security but trying to code constant time\)



In [1]:
import hashlib

def keyGen(n=512):
    """Generate RSA keys (N, p, q, e, d) with modulus N (bit size n)"""
    # Generate two distinct primes p and q of n/2 bits each
    lower = 2^(n//2 - 1)
    upper = 2^(n//2) - 1
    p = random_prime(upper, lbound=lower)
    q = random_prime(upper, lbound=lower)
    while p == q:
        q = random_prime(upper, lbound=lower)
    Nn = p * q
    phi_N = (p-1)*(q-1)
    
    # Choose e such that gcd(e, phi_N) = 1
    e = ZZ.random_element(2, phi_N)
    while gcd(e, phi_N) != 1:
        e = ZZ.random_element(2, phi_N)
    d = inverse_mod(e, phi_N)
    return Nn, p, q, e, d



In [2]:
def encrypt(m, Nn, e):
    """Encrypt message m using public key (N, e)."""
    return pow(m, e, Nn)

def decrypt(c, Nn, d):
    """Decrypt ciphertext c using private key (N, d)."""
    return pow(c, d, Nn)

def checkEnc():
    """Verify encryption/decryption correctness."""
    Nn, p, q, e, d = keyGen()
    m = ZZ.random_element(Nn)
    c = encrypt(m, Nn, e)
    m_decrypted = decrypt(c, Nn, d)
    assert m_decrypted == m, "Decryption failed"
    print("Encryption check passed.")



In [3]:
checkEnc()

Encryption check passed.


In [4]:
def sha1_hash(m):
    """Compute SHA-1 hash of string m and return hex digest."""
    h = hashlib.sha1()
    h.update(m.encode("utf-8"))
    return h.hexdigest()

def fullHash(m, IN):
    """Compute hash value H(m) with output size IN-4 bits."""
    k = ceil(IN / 160)  # SHA-1 produces 160-bit (40 hex) hashes
    hf = "".join(sha1_hash(m + str(i)) for i in range(k))
    # Truncate to (IN//4 -1) hex chars to get IN-4 bits
    truncated_length = (IN // 4) - 1
    hf = hf[:truncated_length]
    return Integer(hf, base=16)

def sign(m, Nn, d):
    """Sign message m using private key d."""
    IN = Nn.nbits()
    h_m = fullHash(m, IN)
    sigma = pow(h_m, d, Nn)
    return sigma

def verify(sigma, m, Nn, e):
    """Verify signature sigma using public key e."""
    IN = Nn.nbits()
    h_m = fullHash(m, IN)
    h_prime = pow(sigma, e, Nn)
    return h_prime == h_m

def checkSig():
    """Verify signature generation/verification correctness."""
    Nn, p, q, e, d = keyGen()
    m = "message"
    sigma = sign(m, Nn, d)
    assert verify(sigma, m, Nn, e), "Signature verification failed"
    print("Signature check passed.")


In [5]:
checkSig()

Signature check passed.
